In [1]:
import pandas as pd
import numpy as np

## 讀取包裹檔案

In [2]:
df = pd.read_csv("all_table_noquot.csv", encoding="ANSI")
df.head()

,ACC1,ACC2,ACC5,ACC15,ACC17,ACC19,ACC21,ACC22,ACC23,ACC25,...,ACC35_63,ACC35_64,ACC35_65,ACC35_66,ACC35_67,ACC35_nan,ACC36_F,ACC36_R,ACC36_nan,pre
0,"""600001""",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0.229
1,"""540000""",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0.954
2,"""900000""",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0.912
3,"""900000""",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,1.178
4,"""900000""",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0.912


## 選取為潛在客戶的包裹

In [3]:
df = df[df['pre']>=0]

### 利用寄送的支局當作據點

In [4]:
df['part_no'] = df['TT2'].str.slice(7, 13)
df.head()

,ACC1,ACC2,ACC5,ACC15,ACC17,ACC19,ACC21,ACC22,ACC23,ACC25,...,ACC35_64,ACC35_65,ACC35_66,ACC35_67,ACC35_nan,ACC36_F,ACC36_R,ACC36_nan,pre,part_no
0,"""600001""",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0.229,001045
1,"""540000""",0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0.954,540000
2,"""900000""",0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0.912,900000
3,"""900000""",0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,1.178,900000
4,"""900000""",0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0.912,900000


## 以寄送支局為單位計算譴再包裹個數

In [15]:
df1 = pd.DataFrame()
df1 = df.groupby('part_no').count()
df1 = df1['pre']
df1.head()

part_no
000002       1
000009       1
001000    4395
001001    5651
001002    7473
Name: pre, dtype: int64

### 整里程新的df

In [16]:
df1['郵務局號']=df1.index
df1 = df1.to_frame()
df1['郵務局號']=df1.index
df1.head()

,pre,郵務局號
part_no,,
000002,1,000002
000009,1,000009
001000,4395,001000
001001,5651,001001
001002,7473,001002


### 讀取寄送支局的經緯度

In [35]:
part_df = pd.read_csv('郵局據點.csv', encoding='ANSI')
part_df = part_df[['郵務局號','經度','緯度']]
part_df.head()

,郵務局號,經度,緯度
0,100000,121.511431,25.047355
1,100001,121.528922,25.034080
2,100002,121.507005,25.041320
3,100003,121.501031,25.040769
4,100004,121.504565,25.036559


### 以寄送支局為單位進行交集合併

In [36]:
part_df['郵務局號'] = part_df['郵務局號'].astype(str)
join = pd.merge(df1, part_df, left_on='part_no',right_on='郵務局號',how='inner')
join.head()

,pre,郵務局號_x,郵務局號_y,經度,緯度
0,7821,100000,100000,121.511431,25.047355
1,5608,100001,100001,121.528922,25.034080
2,4339,100002,100002,121.507005,25.041320
3,2263,100003,100003,121.501031,25.040769
4,1572,100004,100004,121.504565,25.036559


## -----------------------------------------------------------------------------------------------

In [30]:
import numpy as np
import folium
from folium.plugins import HeatMap

## 以台中為中心畫圖

In [31]:
map_= folium.Map(location=[24.123206,120.675679] , zoom_start=1.5,  width='80%',
                 tiles="OpenStreetMap" , control_scale = True,no_touch = True)#zomm_start
map_.add_child(folium.LatLngPopup()) #為地圖任一地點添加經緯度 

### 畫出泡泡圖

In [32]:
for i  in range(len(join)):
    folium.Circle(
        location = [join['緯度'][i],join['經度'][i]],  #iloc：指矩陣的位置，分dataframe
        radius = join['pre'][i]*0.05,
        color = 'blue',
        fill = True,
        fill_color = 'blue',
        fill_opacity=0.2
    ).add_to(map_)
map_.save('bouble.html')

### 畫出熱力圖

In [51]:
data_heat = join[['緯度', '經度', 'pre']].values.tolist()
hmap = folium.Map(location=[24.123206,120.675679],
                  tiles='OpenStreetMap',control_scale=True, zoom_start=13)
hmap.add_child(HeatMap(data_heat, radius=5, gradient={.3: 'blue', .45: 'lime', .55: 'yellow', .7: 'red'}))
  # 保存为html文件
hmap.save('heat.html')